## Proposed features for weights and biases
Draft Rank = 35% <br>
Projected Season Points = 15% <br>
Positions needing to be filled on the team = 25% <br>
Position Value = 10% <br>
Drafter's Pro Team Preferences = 7% <br>
Player Injury Implications = 6% <br>
Star Power = 4% <br>

In [1]:
import json
import ast
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, MinMaxScaler

In [2]:
# pull in our data from the csv
data = pd.read_csv('analysis/data/draft-players-2023.csv')

In [3]:
# The first column is the original indexes which are not needed, so we will drop it
data = data.drop(data.columns[0], axis=1)
data.head(5)

,draftRank,eligibleSlots,playerId,name,projPoints
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0


In [4]:
all_projected_points = data['projPoints'].sum()
print("The total projected points ESPN has calculated for all players in the 2023 season is:", f'{int(all_projected_points):,}')

The total projected points ESPN has calculated for all players in the 2023 season is: 150,474


## Position analysis
For each position that will be used in our league, we are going to evaluate the strength and value of players in those positions.
The positions our league uses is: Catcher, 1B, 2B, 3B, SS, 3 OF, DH, 4 Utility, and 9 Pitchers
SP are limited to 8 starts per week, so it is best to plan to have 6 SP and 3 RP

We are going to aggregate the following statistics:
- **Position Weight** = Projected fantasy score / Total fantasy point projections
- **Normalized Position Weight** = Position weight on a normalized scale
- **Max Projected Points** = Maximum amount of projected fantasy points for a player in this position
- **Min Projected Points** = The amount of fantasy points the lowest scoring drafted player would be projected to earn.  This is calculated as n = (# of starters for position * # of teams in league).  Ascending sorted projected fantasy players\[n]



In [5]:
# create an object of each position and their ID using the positions lookup file
position_map = {}
with open('lookups/espn-positions-map.json') as f:
  position_map = json.load(f)

In [6]:
# We will manually configure which IDs are being used by our league for positions
# and how many starters we want to have at each position to filter our positions object
STARTER_POSITION_IDS = [0, 1, 2, 3, 4, 5, 11, 14, 15]
POSITIONS_STARTERS_CONFIG = {"0": 1, "1": 1, "2": 1, "3": 1, "4": 1, "5": 3, "11": 1, "14": 6, "15": 3, "12": 4, "16": 5}

position_map = {k: v for k, v in position_map.items() if v['posId'] in STARTER_POSITION_IDS}
# Note: we just removed UTIL and BENCH from the position map, even though they exist in the starters_config dict
# This is appropriate for our up front calculations, but we will still consider this data later

# appending the starters allowed to each position
for k, v in position_map.items():
  position_map[k]['startersAllowed'] = POSITIONS_STARTERS_CONFIG[str(v['posId'])]

In [7]:
# We are going to do some analysis on each positon and update it to the position_map object
for position in STARTER_POSITION_IDS:

  # subset all player data based on who is eligible for the position sorted by projected points
  position_eligible_players = [values for index, values in data.iterrows() if position in ast.literal_eval(values['eligibleSlots'])]
  position_eligible_players = sorted(position_eligible_players, key=lambda k: k['projPoints'], reverse=True)

  # parse the max projected points, min projected points, and total projected points for the position
  max_projected_points = position_eligible_players[0]['projPoints']
  min_projected_points = position_eligible_players[(position_map[str(position)]['startersAllowed']*12)+1]['projPoints']
  sum_projected_points = sum([float(player['projPoints']) for player in position_eligible_players])

  # save each data point to the position_map object
  position_map[str(position)]['totalProjectedPoints'] = sum_projected_points
  position_map[str(position)]['maxProjectedPoints'] = max_projected_points
  position_map[str(position)]['minProjectedPoints'] = min_projected_points

In [8]:
# Let's take a look at an updated entry from the position_map object
position_map['0']

{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5}

In [9]:
# We are going to calculate how many points each position is projected to contribute
# to overall scoring this year and turn that into a weighted score
for k, v in position_map.items():
  v['overallWeight'] = v['totalProjectedPoints'] / all_projected_points

In [10]:
# In order to do accurate analysis, we are also going to observe the normalized weight for each position
pos_points_list = [v['totalProjectedPoints'] for k, v in position_map.items()]
norm_pos_points_list = normalize([pos_points_list])[0]
position_map = [{**v, **{'normalizedPositionValue': norm_pos_points_list[i]}} for i, (k, v) in enumerate(position_map.items())]

In [11]:
print("Total Points:", pos_points_list)
print("Normalized Points:", norm_pos_points_list)
position_map[0]

Total Points: [10139.5, 15608.0, 15170.5, 16141.0, 13826.5, 35857.5, 22173.5, 43800.0, 18474.5]
Normalized Points: [0.14246494 0.21930005 0.21315296 0.22678896 0.1942691  0.50381544
 0.31154854 0.61541146 0.25957578]


{'posId': 0,
 'name': 'Catcher',
 'abbrev': 'C',
 'startersAllowed': 1,
 'totalProjectedPoints': 10139.5,
 'maxProjectedPoints': 452.5,
 'minProjectedPoints': 316.5,
 'overallWeight': 0.0673837340670149,
 'normalizedPositionValue': 0.14246494211383717}

## Evaluate Players:
We are going to weight the following features for individual players:
- What is the rank of the player compared to all projected points in their position
- Handle multiple positions through an averaging algorithm
- What is the normalized |weight of a players projected fantasy points
- How much of a negative impact to other teams does drafting the player create?

In [12]:
# loop through each player in our dataset and calculate their positional value
for index, player in data.iterrows():
  # identify the relevant starter positions for this player
  players_positions = [position for position in ast.literal_eval(player['eligibleSlots']) if position in STARTER_POSITION_IDS]

  # if there are multiple eligible positions, this will average them together
  scores = []
  for pos in players_positions:
    position_stats = [position for position in position_map if position['posId'] == pos]
    score = position_stats[0]['normalizedPositionValue'] * player['projPoints']
    scores.append(score)
  
  # calculate the position-adjusted score for the player based on their eligible positions
  players_weights = [position['overallWeight'] for position in position_map if position['posId'] in players_positions]
  normalized_players_weights = normalize([players_weights])[0]

  avg_score = np.mean(scores) * (len(players_positions) * sum(normalized_players_weights))

  # update the player's positional value in our original dataset
  data.loc[index, 'positionAdjustedPointsScore'] = avg_score

data.head(25)

,draftRank,eligibleSlots,playerId,name,projPoints,positionAdjustedPointsScore
0,1,"[11, 12, 13, 14, 16, 17]",39832,Shohei Ohtani,588.0,732.470167
1,2,"[10, 5, 12, 16, 17]",36969,Juan Soto,602.0,303.296895
2,3,"[13, 14, 16, 17]",32081,Gerrit Cole,629.0,387.093806
3,4,"[3, 7, 19, 11, 12, 16, 17]",32801,Jose Ramirez,676.0,508.392321
4,5,"[13, 14, 16, 17]",39878,Corbin Burnes,613.0,377.247223
5,6,"[1, 7, 19, 11, 12, 16, 17]",35002,Vladimir Guerrero Jr.,595.0,440.090799
6,7,"[1, 7, 19, 12, 16, 17]",30193,Freddie Freeman,607.0,133.115128
7,8,"[9, 10, 5, 11, 12, 16, 17]",33192,Aaron Judge,612.0,686.858738
8,9,"[13, 14, 16, 17]",28976,Max Scherzer,510.0,313.859843
9,10,"[10, 5, 12, 16, 17]",33039,Mookie Betts,559.5,281.884739


## Determine Position Needs
- We will need to consider what fielding positions we still need on our team
- This should consider what positions we've already picked
- This should consider how much value is still left on the draft board for a given position at this moment in time
- This should consider how much value will still be left on the board by time we get to the next pick
* A super feature would be to predict, out of the most probable remaining draft pick outcomes from the current pick on, by all teams, which player at what pick delivers me the most value

In [13]:
# First, we are going to define the draft order for this season
draft_order_ids_list = []

# We will use the draft-order.json file which describes the order that Fantasy teams will pick in the draft
with open('analysis/data/draft-order-2023.json', 'r') as f:
  draft_order = json.load(f)
  draft_order = sorted(draft_order, key=lambda k: k['draftOrder'])
  print("The draft order is:", [player['teamName'] for player in draft_order])
  rounds = 27

  # We will loop through the draft order and add the team name to the list for each pick
  # There will be some conditions applied to account for the "snake" draft order pattern
  for i in range(rounds):
    reverse_toggle = i % 2

    if i < 3:
      reverse_toggle = 1

    if reverse_toggle == 1:
      for player in draft_order:
        draft_order_ids_list.append(player['teamId'])
    elif reverse_toggle == 0:
      for player in draft_order[::-1]:
        draft_order_ids_list.append(player['teamId'])

print("There will be", len(draft_order_ids_list), "picks over", int(len(draft_order_ids_list) / len(draft_order)), "rounds")

The draft order is: ['BG 420', 'The Dissenters', "Loser's Club L", 'RIP Paulie Walnuts', 'Nomar Losing', 'Pecan Sandies', 'Rocket City Trash Pandas', '2-Time Champ Booch', 'Commissioner Getbot420', 'Big Trains', 'Hot Italian Snausages', 'Vandelay Industries']
There will be 324 picks over 27 rounds


In [14]:
# Hold all the owner info in an array
owners = []
with open ('analysis/data/owners.json') as f:
  owners = json.load(f)

# Update the owners object to include the positions they need to fill
for owner in owners:
  owner['posNeeds'] = {"0": 1, "1": 1, "2": 1, "3": 1, "4": 1, "5": 1, "11": 1, "14": 1, "15": 1}

Formula for calculating position needs:
$$
  \int \frac{StartersNeeded - \frac{1}{PlayersEligiblePositions}}{RosterSize -(PickNumber - 1)}
$$

In [15]:
def updatePositionNeeds(picker: str, owner_list: list, pick: dict):
  """
  This function will generate a pick for the given picker
  Parameters:
    picker (str): The name of the team that is picking
    owner_list (list): The list of owners with their current positional needs
    pick (dict): Details about the player that was just drafted
  """

  pos_scores = [owner['posNeeds'] for owner in owner_list if owner['teamId'] == picker]

  matching_positions = [position for position in ast.literal_eval(pick['eligibleSlots']) if int(position) in STARTER_POSITION_IDS]
  positions_player_can_fill = sum([v for k, v in POSITIONS_STARTERS_CONFIG.items() if int(k) in matching_positions]) + POSITIONS_STARTERS_CONFIG['16']
  for pos in matching_positions:
    players_eligible_positions = len(matching_positions)
    
    if pos in [0, 1, 2, 3, 4, 5, 11, 12]:
      starters_needed = positions_player_can_fill + POSITIONS_STARTERS_CONFIG['12']
    else:
      starters_needed = positions_player_can_fill
    
    teams_pick_number = int(np.ceil(pick['pickNumber'] / 12))
    result = (starters_needed - (1 / players_eligible_positions)) / (27 - (teams_pick_number - 1))
    new_score = pos_scores[0][str(pos)] * result
    # print("Old score:", pos_scores[0][str(pos)], "\nNew score:", new_score)
    owner_list['teamId' == picker]['posNeeds'][str(pos)] = new_score

In [16]:
for owner in owners:
  owner['picks'] = []

In [17]:
draft_list_subset = draft_order_ids_list[:20]

In [18]:
owners

[{'ownerId': '{09B45A77-9B6D-4F38-91ED-8770EB8E1B39}',
  'ownerName': 'Zack Bessette',
  'teamId': 1,
  'teamName': 'BG 420',
  'posNeeds': {'0': 1,
   '1': 1,
   '2': 1,
   '3': 1,
   '4': 1,
   '5': 1,
   '11': 1,
   '14': 1,
   '15': 1},
  'picks': []},
 {'ownerId': '{3A5C6608-D4CA-4A22-9C66-08D4CAFA2282}',
  'ownerName': 'Kevin Boyles',
  'teamId': 2,
  'teamName': 'Vandelay Industries',
  'posNeeds': {'0': 1,
   '1': 1,
   '2': 1,
   '3': 1,
   '4': 1,
   '5': 1,
   '11': 1,
   '14': 1,
   '15': 1},
  'picks': []},
 {'ownerId': '{DD7CBFDF-7F2F-4621-9E9F-6AB906B0A4E0}',
  'ownerName': 'Kyle Booth',
  'teamId': 3,
  'teamName': 'Big Trains',
  'posNeeds': {'0': 1,
   '1': 1,
   '2': 1,
   '3': 1,
   '4': 1,
   '5': 1,
   '11': 1,
   '14': 1,
   '15': 1},
  'picks': []},
 {'ownerId': '{496167B6-CD2E-43D3-A167-B6CD2E13D3DC}',
  'ownerName': 'Ant Barberio',
  'teamId': 4,
  'teamName': 'RIP PAULIE WALNUTS',
  'posNeeds': {'0': 1,
   '1': 1,
   '2': 1,
   '3': 1,
   '4': 1,
   '5': 1,
 

In [21]:
def generatePick(picker: str, pick_number: int, owner_list: list, current_pick_list: list, remaining_picks: pd.DataFrame):
  """
  This function will generate a pick for the given picker
  """
  PICK_ROUND = np.ceil(pick_number / 12)
  TOTAL_PICKS = 27 * 12
  BATTER_ELIGIBLE_POSITIONS = [0, 1, 2, 3, 4, 5, 11, 12]
  PITCHER_ELIGIBLE_POSITIONS = [14, 15]

  draft_rank_score = TOTAL_PICKS - pick_number

  # apply min max scaling to the draft rank score
  for index, player in remaining_picks.iterrows():
    remaining_picks.loc[index, 'draftRankScore'] = TOTAL_PICKS - player['draftRank']

  normalized_draft_rank_score = MinMaxScaler(feature_range=(-1, 1)).fit_transform(np.array(remaining_picks[['draftRankScore']]).reshape(-1, 1)).reshape(-1)
  remaining_picks['draftRankScore'] = normalized_draft_rank_score.tolist()

  # normalize the position adjusted score
  normalized_position_adjusted_score = normalize(np.array(remaining_picks[['positionAdjustedPointsScore']]).reshape(-1, 1), axis=0, norm='max').reshape(-1)
  remaining_picks['positionAdjustedScore'] = normalized_position_adjusted_score.tolist()

  # calculate the position needs score
  for index, player in remaining_picks.iterrows():
    matching_positions = [position for position in ast.literal_eval(player['eligibleSlots']) if int(position) in STARTER_POSITION_IDS]
    owner = [owner for owner in owner_list if owner['teamId'] == picker][0]
    positions_needs_values = [v for k, v in owner['posNeeds'].items() if int(k) in matching_positions]
    positions_needs_score = sum(positions_needs_values) / len(positions_needs_values)
    remaining_picks.loc[index, 'positionNeedsScore'] = positions_needs_score

  for index, player in remaining_picks.iterrows():
    remaining_picks.loc[index, 'overallScore'] = (player['draftRankScore'] + player['positionAdjustedScore'] + player['positionNeedsScore']) / 3

  best_pick = remaining_picks.sort_values(by=['overallScore'], ascending=False).iloc[0]

  payload = {
    'pickNumber': pick_number,
    'pickRound': PICK_ROUND,
    'teamName': [owner['teamName'] for owner in owner_list if owner['teamId'] == picker][0],
    'teamId': picker,
    'playerId': best_pick['playerId'],
    'playerName': best_pick['name'],
    'draftRank': best_pick['draftRank'],
    'eligibleSlots': best_pick['eligibleSlots'],
    'projPoints': best_pick['projPoints'],
    'overallScore': best_pick['overallScore'],
  }

  return payload

In [22]:
pick_number = 1
drafted_players = []
for picker in draft_list_subset:
  remaining_players = data[~data['playerId'].isin(drafted_players)]
  drafted_player = generatePick(picker, pick_number, owners, drafted_players, remaining_players)
  print(drafted_player)
  updatePositionNeeds(picker, owners, drafted_player)
  owners['teamName' == picker]['picks'].append(drafted_player)
  drafted_players.append(drafted_player)
  pick_number += 1
print(drafted_players)

{'pickNumber': 1, 'pickRound': 1.0, 'teamName': 'BG 420', 'teamId': 1, 'playerId': 33192, 'playerName': 'Aaron Judge', 'draftRank': 8, 'eligibleSlots': '[9, 10, 5, 11, 12, 16, 17]', 'projPoints': 612.0, 'overallScore': 0.9110089307096919}
{'pickNumber': 2, 'pickRound': 1.0, 'teamName': 'The Dissenters', 'teamId': 7, 'playerId': 39832, 'playerName': 'Shohei Ohtani', 'draftRank': 1, 'eligibleSlots': '[11, 12, 13, 14, 16, 17]', 'projPoints': 588.0, 'overallScore': 1.0}
{'pickNumber': 3, 'pickRound': 1.0, 'teamName': "Loser's Club L", 'teamId': 5, 'playerId': 39832, 'playerName': 'Shohei Ohtani', 'draftRank': 1, 'eligibleSlots': '[11, 12, 13, 14, 16, 17]', 'projPoints': 588.0, 'overallScore': 1.0}
{'pickNumber': 4, 'pickRound': 1.0, 'teamName': 'RIP PAULIE WALNUTS', 'teamId': 4, 'playerId': 39832, 'playerName': 'Shohei Ohtani', 'draftRank': 1, 'eligibleSlots': '[11, 12, 13, 14, 16, 17]', 'projPoints': 588.0, 'overallScore': 1.0}
{'pickNumber': 5, 'pickRound': 1.0, 'teamName': 'Nomar Losing

In [27]:
remaining_players = data[drafted_players['playerId'].isin(drafted_players)]
remaining_players

,draftRank,eligibleSlots,playerId,name,projPoints,positionAdjustedPointsScore


In [ ]:
pick_number = 1
for picker in draft_order_ids_list:
  for team in predicted_draft_picks:
    if team['teamName'] == picker:
      pick_index = generatePick(picker, pick_number, predicted_draft_picks, posEligData, data)
      team['picks'].append(posEligData.iloc[pick_index][['playerId', 'name']].to_dict())
      posEligData = posEligData.drop(pick_index)
      pick_number += 1
      team['picks'] = sorted(team['picks'], key=lambda k: k['playerId'])


In [ ]:
formula = sum({k: v for k, v in position_starters_config.items() if k in batter_pos}.values()) - sum({pick['name'] for pick in current_pick_list}.values())

In [ ]:
criteria_weights = pd.DataFrame(
  [
    ['draftRank', 1, 1, 1, 1, 1, 1, 1, 0.35],
    ['projPoints', 1, 1, 1, 1, 1, 1, 1, 0.15],
    ['posNeeds', 1, 1, 1, 1, 1, 1, 1, 0.25],
    ['posValue', 1, 1, 1, 1, 1, 1, 1, 0.1],
    ['favTeam', 1, 1, 1, 1, 1, 1, 1, 0.07],
    ['injuryFactor', 1, 1, 1, 1, 1, 1, 1, 0.06],
    ['starPower', 1, 1, 1, 1, 1, 1, 1, 0.02],
  ],
  columns = ['draftRank', 'projPoints', 'posNeeds', 'posValue', 'favTeam', 'injuryFactor', 'starPower', 'priority']
)
priority_ranks = {'draftRank': 0.35, 'projPoints': 0.15, 'posNeeds': 0.25, 'posValue': 0.1, 'favTeam': 0.07, 'injuryFactor': 0.06, 'starPower': 0.02}

In [ ]:
def calcDraftRankScore(rank: int, total_picks = 324):
  score = total_picks - rank
  return